<a href="https://colab.research.google.com/github/ricardogr07/purkinje-uv/blob/feat%2Fchange-parameters-to-a-dataclass/examples/minimal_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purkinje-UV: Ellipsoid Demo

This notebook demonstrates the full workflow:

**mesh → growth → Purkinje tree → activation → save → visualize**

You’ll end with a VTU file containing activation times, plus an inline visualization.

**Flow**
1) Clone the `purkinje-uv` repo (just to get a known `data/ellipsoid.obj` path)  
2) Install the local checkout (so we use *that* code)  
3) Import libraries & configure logging  
4) Define `FractalTreeParameters`  
5) Grow the tree  
6) Build `PurkinjeTree`, run FIM activation, save VTU  
7) Validate output with `meshio`, visualize inline with PyVista


### Installation
Use one of the following:

**From PyPI (if available)**
```bash
pip install purkinje_uv numpy meshio pyvista pyvistaqt

In [ ]:
# Install core runtime dependencies.
# NOTE: If you already have these locally, you can skip this cell.
%pip -q install --upgrade pip
%pip -q install numpy meshio pyvista pyvistaqt

## Download or reuse the `purkinje-uv` repository

We clone the repository so we can reference `data/ellipsoid.obj` with a known relative path.  
The cell will try `git clone` first, then fall back to a ZIP download if needed.


In [ ]:
from pathlib import Path

REPO_URL = "https://github.com/ricardogr07/purkinje-uv"
DEST_DIR = Path("external/purkinje-uv")

if not DEST_DIR.exists():
    DEST_DIR.parent.mkdir(parents=True, exist_ok=True)
    print("Cloning purkinje-uv…")
    !git clone --depth 1 {REPO_URL} {str(DEST_DIR)}
else:
    print("Using existing checkout:", DEST_DIR)

REPO_ROOT = DEST_DIR.resolve()
MESH_PATH = REPO_ROOT / "data" / "ellipsoid.obj"
assert MESH_PATH.exists(), f"Expected mesh not found at: {MESH_PATH}"

print("Repo root:", REPO_ROOT)
print("Mesh path:", MESH_PATH)

In [ ]:
# Install the local repo
!pip -q install -e {str(REPO_ROOT)}

# Ensure runtime dependencies are present (matches demo flow)
!pip -q install numpy meshio pyvista pyvistaqt pythreejs

In [ ]:
from pathlib import Path

print("REPO_ROOT:", REPO_ROOT)
print("Top-level files/dirs:")
!ls -la {str(REPO_ROOT)}
print("\nLikely src layout (if present):")
!ls -la {str(REPO_ROOT / 'src')}

In [ ]:
import sys
from pathlib import Path

# If the editable install didn’t take, fall back to adding src/ manually.
src_path = REPO_ROOT / "src"
pkg_root = src_path if src_path.exists() else REPO_ROOT

if str(pkg_root) not in sys.path:
    sys.path.insert(0, str(pkg_root))

import importlib
import purkinje_uv

print("purkinje_uv loaded from:", purkinje_uv.__file__)

# Now import the symbols you need
from purkinje_uv import FractalTree, FractalTreeParameters, PurkinjeTree
print("Symbol imports OK.")


In [ ]:
from __future__ import annotations

import logging
from pathlib import Path

import numpy as np
import meshio
import pyvista as pv

from purkinje_uv import FractalTree, FractalTreeParameters, PurkinjeTree

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("purkinje_uv.examples.ellipsoid_demo")

print("Imports OK.")

## Locate mesh file
The examples assume an ellipsoid mesh at `data/ellipsoid.obj`.

In [ ]:
meshfile = Path(MESH_PATH)
print(f"Using mesh: {MESH_PATH}")

## Configure parameters

`FractalTree` reads `meshfile` from the parameters and computes UV scaling internally.  
We serialize parameters to JSON for reproducibility.

In [ ]:
lseg = 0.01
p = FractalTreeParameters(
    meshfile=str(MESH_PATH),
    init_node_id=738,
    second_node_id=210,
    l_segment=lseg,
    init_length=0.3,
    length=0.15,
    fascicles_length=[20 * lseg, 40 * lseg],
    fascicles_angles=[-0.4, 0.5],  # radians
)

params_json = REPO_ROOT / "examples" / "params.json"
params_json.parent.mkdir(parents=True, exist_ok=True)
p.to_json_file(str(params_json))

print("Saved params to:", params_json)

## Grow fractal tree on the surface

In [ ]:
tree = FractalTree(p)
tree.grow_tree()

print(
    f"Tree grown → nodes={len(tree.nodes_xyz)}, "
    f"segments={len(tree.connectivity)}, end_nodes={len(tree.end_nodes)}"
)

## Build Purkinje tree, run activation (FIM), save VTU, and sanity-check with `meshio`

In [ ]:
Ptree = PurkinjeTree(
    np.asarray(tree.nodes_xyz),
    np.asarray(tree.connectivity),
    np.asarray(tree.end_nodes),
)

act = Ptree.activate_fim([0], [0.0], return_only_pmj=False)
pmj = Ptree.pmj

assert isinstance(act, np.ndarray) and act.ndim == 1
assert pmj is not None and np.all((pmj >= 0) & (pmj < act.shape[0]))
print(f"Activation computed → length={act.shape[0]}, PMJs={pmj.size}")

out_dir = REPO_ROOT / "examples" / "output"
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / "ellipsoid_purkinje_AT.vtu"
Ptree.save(str(out_file))
print("Saved:", out_file)

m = meshio.read(str(out_file))
print(
    f"meshio OK → points={m.points.shape[0]}, cells={sum(len(c.data) for c in m.cells)}"
)

## Visualize with PyVista (inline)

We attempt to use the `pythreejs` backend for inline rendering.  
If the backend is unavailable, we’ll save a screenshot instead.

In [ ]:
%pip -q install ipywidgets pythreejs
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
%pip -q install "trame>=3" trame-vtk trame-vuetify

In [ ]:
# --- Colab-safe PyVista visualization (HTML backend + embedded fallback) ---

import pyvista as pv
from IPython.display import HTML, display

def pick_scalar(ds):
    preferred = ["Activation", "AT", "activation_time", "time_activation"]
    for name in preferred:
        if name in ds.array_names:
            return name
    if ds.point_data:
        return list(ds.point_data.keys())[0]
    if ds.cell_data:
        return list(ds.cell_data.keys())[0]
    return None

def to_tubes(ds, radius=0.003):
    try:
        return ds.tube(radius=radius)
    except Exception:
        return ds

ds = pv.read(str(out_file))
scal_name = pick_scalar(ds)
vis = to_tubes(ds, radius=0.003)

# 1) Prefer the pure-HTML backend (no localhost/server)
try:
    pv.global_theme.jupyter_backend = "html"   # valid backends: "static","client","server","trame","html","none"
    print("Using PyVista backend:", pv.global_theme.jupyter_backend)

    p = pv.Plotter(notebook=True)
    p.add_mesh(
        vis,
        scalars=scal_name if scal_name else None,
        show_scalar_bar=bool(scal_name),
        scalar_bar_args={"title": scal_name or ""},
    )
    p.show(title="Purkinje VTU (ellipsoid)")   # should render right in the cell
except Exception as e_html_backend:
    print("Direct HTML backend render failed:", e_html_backend)

    # 2) Self-contained HTML export and embed (no web server)
    try:
        html_path = out_dir / "ellipsoid_view.html"

        # Use an offscreen renderer to guarantee scene is built
        try:
            pv.start_xvfb()  # no-op if not needed
        except Exception:
            pass

        p = pv.Plotter(off_screen=True)
        p.add_mesh(
            vis,
            scalars=scal_name if scal_name else None,
            show_scalar_bar=bool(scal_name),
            scalar_bar_args={"title": scal_name or ""},
        )
        # export a standalone HTML (threejs) and embed its contents
        p.export_html(str(html_path))
        display(HTML(html_path.read_text(encoding="utf-8")))
        print("Embedded exported HTML:", html_path)
    except Exception as e_export:
        print("HTML export/embed failed:", e_export)

        # 3) Last resort: save a screenshot and display it
        try:
            img_path = out_dir / "ellipsoid_render.png"
            p = pv.Plotter(off_screen=True)
            p.add_mesh(
                vis,
                scalars=scal_name if scal_name else None,
                show_scalar_bar=bool(scal_name),
                scalar_bar_args={"title": scal_name or ""},
            )
            p.show(screenshot=str(img_path))
            from IPython.display import Image
            display(Image(filename=str(img_path)))
            print("Screenshot fallback saved:", img_path)
        except Exception as e_ss:
            print("Screenshot fallback failed:", e_ss)